In [74]:
import numpy as np
import pandas
from pandas.core.window.indexers import BaseIndexer


In [75]:

class CustomIndexer(BaseIndexer):
    def get_window_bounds(self, num_values=0, min_periods=None, center=None, closed=None):
        end = np.arange(0, num_values, dtype="int64")
        end += 4
        start = end - 3

        end = np.clip(end, 0, num_values)
        start = np.clip(start, 0, num_values)

        return start, end


def concat_rows(df, n):
    new_cols = [
        f"{col}{idx}"
        for idx in range(1, n + 1)
        for col in df.columns
    ]
    n_cols = len(df.columns)
    new_df = pandas.DataFrame(
        df.values.reshape([-1, n_cols * n]),
        columns=new_cols
    )
    return new_df

In [76]:
filename = "data/2021_LoL_esports_match_data_from_OraclesElixir_20210515.csv"

data = pandas.read_csv(filename)
data = data.reindex(index=data.index[::-1]).reset_index()

In [77]:
team_columns = ["date", "actual_result", "playerid", "gameid", "team", "gamelength", "result", "dragons", "barons",
                "riftheralds", "towers"]
player_columns = ["date", "player", "gameid", "kills", "deaths", "assists", "dpm", "damageshare",
                  "damagetakenperminute", "wpm",
                  "vspm", "earned gpm", "cspm", "csat10", "goldat10", "killsat10", "deathsat10", "assistsat10",
                  "csat15", "goldat15", "killsat15", "deathsat15", "assistsat15"]


In [78]:
data = data.sort_values(by=["date", "team"])
data = data.reset_index(drop=True)
data["actual_result"] = data["result"]

In [79]:


indexer = CustomIndexer(window_size=1)

player_data = (
    data
        .filter(player_columns)
        .groupby(pandas.Grouper(key="player"))
        .rolling(window=indexer, min_periods=1, on="gameid")
        .mean()
        .reset_index()
        .rename(columns={"level_1": "id"})
        .sort_values(by="id")
        .reset_index()
        .drop(columns=["index", "player", "id", "gameid"])
)



In [80]:
team_data = (
    data
        .query("playerid > 10")
        .filter(team_columns)
        .groupby(pandas.Grouper(key="team"))
        .rolling(window=indexer, min_periods=1, on="actual_result")
        .mean()
        .reset_index()
        .rename(columns={"level_1": "id"})
        .sort_values(by="id")
        .reset_index()
        .drop(columns=["index", "playerid", "id"])
)

In [81]:
game_data_player = concat_rows(player_data, 10)
game_data_team = concat_rows(team_data, 2)
game_data_team.drop(columns=["actual_result2", "team1", "team2"], inplace=True)

game_data = (
    pandas
        .concat([game_data_team, game_data_player], axis=1)
        .dropna()
)


game_result = game_data["actual_result1"]
game_data.drop(columns=["actual_result1"], inplace=True)


In [82]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier



from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(game_result)


trainX, testX, trainY, testY = train_test_split(
    game_data, encoded, test_size=.33)
model = RidgeClassifier().fit(trainX, trainY)

print(model.score(trainX, trainY))
print(model.score(testX, testY))


# np.savetxt("aggr_data.csv", data_aggr)

0.6496124031007752
0.5997482693517936


In [83]:
import os
import torch
import torch.nn as nn

from torch.utils.data import TensorDataset, DataLoader


def create_dataset(input_data):
    #random.shuffle(input_data)

    target = torch.tensor(game_result, dtype=torch.long)
    #print(input_data[0,:])
    input_data = torch.tensor(input_data.values.astype(np.float32))
   # input_data = torch.tensor(input_data, dtype=torch.float32)
    #target = torch.tensor(input_data, dtype=torch.long)

    dataset = TensorDataset(input_data, target)

    # Compute batch sizes
    size = len(input_data)
    p1 = int(size * .8)
    p2 = int(size * .1)
    p3 = size - p1 - p2

    train, validation, test = torch.utils.data.random_split(dataset, (p1, p2, p3))

    tra_loader = DataLoader(train, batch_size=64, shuffle=True)
    val_loader = DataLoader(validation, batch_size=128)
    tes_loader = DataLoader(test, batch_size=128)



    return tra_loader, val_loader, tes_loader

In [84]:
import os
import torch
import torch.nn as nn

from torch.utils.data import TensorDataset, DataLoader




class TestNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.seq = nn.Sequential(nn.Linear(input_size, 128),
                        # nn.BatchNorm1d(64),
                        # nn.ReLU(),
                        # nn.Dropout(0.5),
                        nn.Linear(128, 64),
                        # nn.BatchNorm1d(32),
                        # nn.ReLU(),
                        # nn.Dropout(0.5),
                        nn.Linear(64, 32),
                        nn.Linear(32, 16),
                        # nn.BatchNorm1d(16),
                        # nn.ReLU(),
                        nn.Linear(16, output_size))


        def weight_init(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

        self.seq.apply(weight_init)

    def forward(self, x):
        out = self.seq(x)
        return out




def training(train: DataLoader, validation: DataLoader):
    epochs = 150

    model = TestNetwork(game_data.shape[1], 2).float()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()


    # model.train()
    for e in range(epochs):
        val_loss = []
        val_acc = []
        model.train()
        for i, (inputs, target) in enumerate(train):
            inputs, target = inputs.to("cpu"), target.to("cpu")
            optimizer.zero_grad()
            pred_target = model(inputs)
            #pred_target = (pred_target > 0.5).float()
            loss = criterion(pred_target, target.flatten())
            loss.backward()

            val_loss.append(loss.item())

            pred_target = pred_target.detach()
            _, pred_target = torch.max(pred_target, dim=1)
            correct = (pred_target == target.flatten()).sum().item()
            acc = correct / len(target)
            val_acc.append(acc)

            optimizer.step()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)

        # print("TRAIN LOSS", np.mean(val_loss))
        # print("TRAIN ACC", np.mean(val_acc))

        val_loss = []
        val_acc = []
        with torch.no_grad():
            model.eval()
            for i, (input, target) in enumerate(validation):
                input, target = input.to("cpu"), target.to("cpu")
                pred_target = model(input)

                loss = criterion(pred_target, target.flatten().long())

                #pred_target = pred_target[pred_target > 0.5]

                _, pred_target = torch.max(pred_target, dim=1)
                correct = (pred_target == target.flatten()).sum().item()
                acc = correct / len(target)

                val_loss.append(loss.item())
                val_acc.append(acc)

    return model

def testing(model, test: DataLoader):
    acc_list = []
    model.eval()
    with torch.no_grad():
        for i, (input, target) in enumerate(test):
            input, target = input.to("cpu"), target.to("cpu")
            pred_target = model(input)

            _, pred_target = torch.max(pred_target, dim=1)
            correct = (pred_target == target.flatten()).sum().item()
            acc = correct / len(target)
            acc_list.append(acc)
    print('test acc', np.mean(acc))


In [85]:
train, validate, test = create_dataset(game_data)
model = training(train, validate)
testing(model, test)



test acc 0.5714285714285714
